# Naive Bayesian Algorithm
---
## Bayesian Algorithm

Formula 
$$
P(A \vert B) = \frac{P(B \vert A)}{P(B)} \cdot P(A)
$$

where : 

- $P(A)$ : prior
- $P(B|A)$ ：likelihood
- $P(A|B)$ : posterior
- $P(B)$ : envidence

It is natural that 
$$P(A|B) \propto P(B|A)\,P(A)
$$

But what is $P(B)$ for ? 

Let's do some tranformation
$$
P(A \vert B) \cdot P(B) = P(B \vert A) \cdot P(A)
$$

Here we can see that $left = P(AB) = right$ , this statement is true

So $P(B)$ is used to *Probability Normalization*

---
## Naive Bayesian

Assume that we have a collection $\small{X = \{x_1, x_2, \ldots, x_n\}}$ and a class $\small{C}$ , under the condition of **independency** , we have:
$$
P(X \vert C) = P(x_1 \vert C) \cdot P(x_2 \vert C) \cdot \ldots \cdot P(x_n \vert C)
$$



---
## Apply the algorithm

### 1)Train
1. Calculate the `Prior`

$$
P(C_{i}) = \frac{n_{i}}{n}
$$

2. Calculate `likelihood`
$$P(x_{j} \vert C_{i}) = \frac{n_{i,j}}{n_{i}}
$$

### 2)Predict


Given an unsorted set $X$, whose *feature*` are $[x_1 , x_2, ... , x_d]$ . We can calculate the **posterior** probability for each class using Bayes’ theorem:

$$
P(C_{i} \mid X) = \frac{P(X \mid C_{i})}{P(X)} \cdot P(C_{i})
$$


($P(C_{i} \mid X)$ stands for the chance that $X$ belongs to class $C_i$)

&nbsp;

Here, $P(X)$ is a constant with respect to the class $C_i$ , so we can ignore it when comparing classes.  

Now, by applying the **independence assumption** (as in Naive Bayes), the likelihood term $P(X \mid C_i)$ can be factorized into the product of individual feature likelihoods:

$$
P(C_{i} \mid X) \propto P(C_{i}) \cdot P(x_1 \mid C_{i}) \cdot P(x_2 \mid C_{i}) \cdot \ldots \cdot P(x_d \mid C_{i})
$$


( $P(x_j \mid C_{i})$ : the probability that the $j$-th feature takes value $x_j$        given class $C_i$ )

&nbsp;


This shows that the posterior is proportional to the prior of the class multiplied by the product of the conditional probabilities of each feature given the class.

When ***posterior*** reaches the peak in class $C_i$ , we say that $X$ belongs to $C_i$


### 3) Code
Use *iris* as example too

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=3)

In training process , we need to obtain the *label* and corresponding *prior* of every class.
In addition , we gonna calculate the *likelihood* using the **naive assumption**

ps. we need to do some discrete processing


In [3]:
import numpy as np
import pandas as pd


def naive_bayes_fit(X, y):
    """
    :param X: sample features
    :param y: sample labels
    :returns: tuple (prior, likelihoods)
    """
    # calculate prior probabilities for each class
    clazz_labels, clazz_counts = np.unique(y, return_counts=True)
    prior_probs = pd.Series({k: v / y.size for k, v in zip(clazz_labels, clazz_counts)})
    
    # make a copy of X
    X = np.copy(X)
    
    likelihoods = {}
    for j in range(X.shape[1]):  # loop over features , shape [1] stands for the columes (number of features)
        
        # discretize the feature values using equal-width binning and relabel them to 1~5
        X[:, j] = pd.cut(X[:, j], bins=5, labels=np.arange(1, 6))
        
        for i in prior_probs.index:  # loop over classes
            # filter samples by class(i)  and collect the values of this feature(j)
            x_prime = X[y == i, j]
            
            # count the frequency of each unique feature value
            x_values, x_counts = np.unique(x_prime, return_counts=True)
            
            for k, value in enumerate(x_values):  # loop over unique feature values
                # compute the likelihood and store it in a dictionary
                # the dictionary key is a tuple (class, feature index, feature value)
                likelihoods[(i, j, value)] = x_counts[k] / x_prime.size
    return prior_probs, likelihoods


by calling the function , we get a binary array ,
which contains the `prior` of class $\small{C_{i}}$  and the `likelihood` of  the $\small{j}$th featurein class $\small{C_{i}}$ owning`value`
（we used `cut` function in pandas to cut `value` into `1` to `5`）
the former one is a  `Series` object , while the later one is a `dict` object , as is shown below:


In [11]:
p_ci, p_x_ci = naive_bayes_fit(X_train, y_train)
print('prior: ', p_ci, sep='\n')
print('likelihood: ', p_x_ci, sep='\n')

prior: 
0    0.333333
1    0.333333
2    0.333333
dtype: float64
likelihood: 
{(0, 0, np.float64(1.0)): np.float64(0.525), (0, 0, np.float64(2.0)): np.float64(0.45), (0, 0, np.float64(3.0)): np.float64(0.025), (1, 0, np.float64(1.0)): np.float64(0.05), (1, 0, np.float64(2.0)): np.float64(0.375), (1, 0, np.float64(3.0)): np.float64(0.425), (1, 0, np.float64(4.0)): np.float64(0.15), (2, 0, np.float64(1.0)): np.float64(0.025), (2, 0, np.float64(2.0)): np.float64(0.025), (2, 0, np.float64(3.0)): np.float64(0.45), (2, 0, np.float64(4.0)): np.float64(0.3), (2, 0, np.float64(5.0)): np.float64(0.2), (0, 1, np.float64(1.0)): np.float64(0.025), (0, 1, np.float64(3.0)): np.float64(0.325), (0, 1, np.float64(4.0)): np.float64(0.45), (0, 1, np.float64(5.0)): np.float64(0.2), (1, 1, np.float64(1.0)): np.float64(0.175), (1, 1, np.float64(2.0)): np.float64(0.325), (1, 1, np.float64(3.0)): np.float64(0.475), (1, 1, np.float64(4.0)): np.float64(0.025), (2, 1, np.float64(1.0)): np.float64(0.025), (2, 1, n


illustration :


take a look into the `likelihood` , first element 

`(0,0,1): 0.525` means that in class $C_0$ , the posibility of the no.$0$ feature having value `1` equals to `0.525`

---

Next we gonna use the result of the function to predict

In [12]:
def naive_bayes_predict(X, p_ci, p_x_ci):
    """
    naive_bayes predict
    :param X: sample feature
    :param p_ci: prior
    :param p_x_ci: likelihood
    :return: predicted value
    """
    # discretize
    X = np.copy(X)
    for j in range(X.shape[1]):
        X[:, j] = pd.cut(X[:, j], bins=5, labels=np.arange(1, 6))
        
    # 2D array used to store posterior
    # it's shape is (num of test objects , size of class)
    results = np.zeros((X.shape[0], p_ci.size))
    clazz_labels = p_ci.index.values
    for k in range(X.shape[0]):
        for i, label in enumerate(clazz_labels):
            
            # get prior (train result)
            prob = p_ci.loc[label]
            
            # calculate posterior
            for j in range(X.shape[1]):
                # default = 0
                prob *= p_x_ci.get((i, j, X[k, j]), 0)
                
            results[k, i] = prob
            # the P ( class_i | X_obj_k)
            
    # choose the predicted label ( maximum )
    return clazz_labels[results.argmax(axis=1)]

In [13]:
y_pred = naive_bayes_predict(X_test, p_ci, p_x_ci)
y_pred == y_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True])

## Addition

for Naive-Bayes  , it is important to choose the classifier according to `feature`

---
---
## Conclusion
 pros : 
1.  ez logic
2. low cost in calculating :  all the needed data has been calculated during training
3. resist to noise and non-relevant features

Cons:

1. assume the independency